In [1]:
import json
import pandas as pd
import numpy as np
## use predicted_user_stance to get data for users in stance in conversations dataset
# predicted_user_stance


df = pd.read_csv("./data/Contentious_pairs_with_GPT_ADA_embeddings.csv")

df['np_target_text_ada_embedding'] = df['target_text_ada_embedding'].apply(eval).apply(np.array)
df['np_response_text_ada_embedding'] = df['response_text_ada_embedding'].apply(eval).apply(np.array)

print(np.shape(df['np_target_text_ada_embedding'][0]))
print(len(df['response_id']))

with open('./data/tweetid_to_users_v2.json', 'r') as f:
    tweetid_to_users = json.load(f)

    
df['target_user'] = df['target_id'].apply(lambda target_id: tweetid_to_users[str(target_id)] if str(target_id) in tweetid_to_users else None)
df['response_user'] = df['response_id'].apply(lambda target_id: tweetid_to_users[str(target_id)] if str(target_id) in tweetid_to_users else None)


df    



(1536,)
3702


,event,response_id,target_id,interaction_type,label,label_expanded,Confidence_Level,response_text,target_text,truncated,response_text_ada_embedding,target_text_ada_embedding,np_target_text_ada_embedding,np_response_text_ada_embedding,target_user,response_user
0,Santa_Fe_Shooting,997626508050157568,997598447376175104,Quote,Support,Implicit_Support,1.0,"Seriously, wtf is wrong with our political sys...",More children have been killed in schools this...,False,"[-0.025092221796512604, -0.013276644051074982,...","[-0.008838048204779625, -0.003347944701090455,...","[-0.008838048204779625, -0.003347944701090455,...","[-0.025092221796512604, -0.013276644051074982,...",321631653,18655355
1,Santa_Fe_Shooting,997575042027458561,997573240380968961,Quote,Denial,Explicit_Denial,1.0,Ma calls BS! https://t.co/bodEWN5Q4C,Former GOP Rep. Jason Chaffetz: 'Politically c...,False,"[-0.033440690487623215, 0.007639116141945124, ...","[-0.027197668328881264, 0.0034390492364764214,...","[-0.027197668328881264, 0.0034390492364764214,...","[-0.033440690487623215, 0.007639116141945124, ...",2467720274,3176702526
2,Santa_Fe_Shooting,997540582846271494,997535659870117888,Quote,Denial,Explicit_Denial,1.0,"On average, there’s one fake stat about school...","On average, that’s one school shooting every w...",False,"[-0.023548724129796028, 0.022269316017627716, ...","[-0.0044773295521736145, -0.014855715446174145...","[-0.0044773295521736145, -0.014855715446174145...","[-0.023548724129796028, 0.022269316017627716, ...",24939978,1646856415
3,General_Terms,1018569947817992192,1017909301040635904,Quote,Denial,Implicit_Denial,1.0,ONE MIGHT BE MADE UP AND NOT REAL. NOT SURE. S...,I’m so confused... - When we were attacked on ...,False,"[0.004989785607904196, -0.004917326848953962, ...","[-0.013004736974835396, -0.01780531369149685, ...","[-0.013004736974835396, -0.01780531369149685, ...","[0.004989785607904196, -0.004917326848953962, ...",2932942216,766475610059317248
4,General_Terms,1019395289575239680,1017919759474622464,Quote,Denial,Explicit_Denial,1.0,"False, adult friendships are M-F, 9-5 https://...",Adult friendships https://t.co/Cn3r9l4pZJ,False,"[0.023379407823085785, 0.010180916637182236, -...","[0.01642121560871601, 0.0012021968141198158, 0...","[0.01642121560871601, 0.0012021968141198158, 0...","[0.023379407823085785, 0.010180916637182236, -...",2909998207,59621769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3697,Santa_Fe_Shooting,998727686062297088,998641852202012672,Quote,Support,Implicit_Support,2.0,"Obviously, this is fake news because France ha...",#LeftistTerrorism #LeftistSedition #WhineAndCh...,False,"[-0.012956563383340836, 0.0036507989279925823,...","[-0.029683180153369904, -0.013163618743419647,...","[-0.029683180153369904, -0.013163618743419647,...","[-0.012956563383340836, 0.0036507989279925823,...",899115298997055491,23194229
3698,Santa_Fe_Shooting,999033162209443841,999032790304854021,Reply,Support,Implicit_Support,2.0,@Goss30Goss There is no such thing as a Consti...,Our founding fathers did not create a country ...,False,"[-0.023475689813494682, 0.022063223645091057, ...","[0.0012622424401342869, 0.0014467427972704172,...","[0.0012622424401342869, 0.0014467427972704172,...","[-0.023475689813494682, 0.022063223645091057, ...",2869858477,807692474684579840
3699,Santa_Fe_Shooting,999044683509714944,999009592867778560,Reply,Denial,Explicit_Denial,2.0,@teamtrace the trace is a @MikeBloomberg shell...,Police confronted the Santa Fe gunman four min...,False,"[-0.0470198318362236, -0.019923418760299683, -...","[-0.01337257120758295, 0.013017261400818825, -...","[-0.01337257120758295, 0.013017261400818825, -...","[-0.0470198318362236, -0.019923418760299683, -...",3243500510,994750018300018688
3700,Santa_Fe_Shooting,999366932024393728,998952797637890048,Quote,Denial,Implicit_Denial,2.0,This must be a tough position for the parents ...,"“My son, to me, is not a criminal, he’s a vict...",False,"[-0.005462398752570152, -0.004

In [2]:
from stance_processor.extract_data_tables import *
from stance_processor.data_extractor import *
from stance_processor.data_loader import *
from stance_processor.cotraining_steps import *
from stance_processor.ConversationsTextSVMClassifier import *
# from stance_processor.ConversationsTextNeuralNetworkClassifier import *

event_seed_tag_labels= { 'Iran_Deal':('../stance_dataset/contentious_data/', 
      { 'thankyoutrump':-1 , 'iranuprising': 1, 'freeiran': 1}, 
      'labeled_users/Iran_Deal_labels_combined.txt' ),

      'Student_Marches':('../stance_dataset/contentious_data/', 
      { 'defendthesecond' : 1, '2ashallnotbeinfringed': 1,'guncontrolnow':-1, 'gunreformnow': -1,
                          'guncontrolnow': -1,  'marchforourlives': -1,'2adefenders': 1, '2ndamendment': 1}, 
      'labeled_users/Student_Marches_labels_combined.txt' ),

      'Santa_Fe_Shooting':('../stance_dataset/contentious_data/', 
      { 'defendthesecond' : 1, '2ashallnotbeinfringed': 1,'guncontrolnow':-1, 'gunreformnow': -1,
                          'guncontrolnow': -1,  'marchforourlives': -1,'2adefenders': 1, '2ndamendment': 1})
    }
        
        
event_name = 'Student_Marches'
# tweets_path = '../stance_dataset/rawdata/fauci_raw/'  # not needed if data extraction is not required
results_path = './results/'
extracted_files_path = '/Users/sumeet_kumar/Library/CloudStorage/OneDrive-IndianSchoolofBusiness/Research_projects/Stance/Stance/stance_dataset/contentious_data/'

seed_tag_labels = { 'defendthesecond' : 1, '2ashallnotbeinfringed': 1,'guncontrolnow':-1, 'gunreformnow': -1,
                   'guncontrolnow': -1,  'marchforourlives': -1,'2adefenders': 1, '2ndamendment': 1}

label_file_name = '/Users/sumeet_kumar/Library/CloudStorage/OneDrive-IndianSchoolofBusiness/Research_projects/Stance/Stance/stance_dataset/labeled_users/' + event_name  + '_labels_combined.txt' #If not, leave blank


seed_tag_labels


{'defendthesecond': 1,
 '2ashallnotbeinfringed': 1,
 'guncontrolnow': -1,
 'gunreformnow': -1,
 'marchforourlives': -1,
 '2adefenders': 1,
 '2ndamendment': 1}

In [3]:
# Read twitter data and create new files (moslty csv and json files)
# dc = DataTablesCreator(tweets_path, event_name, extracted_files_path)
# dc.extract_data()

# Read the newly created files and load them to memory
event_predicted_user_stance = {}
event_predicted_user_stance_conf = {}
event_table_data = {}

for event_name in ['Student_Marches', 'Santa_Fe_Shooting',  'Iran_Deal']:
    
    seed_tag_labels = event_seed_tag_labels[event_name][1]
    print(seed_tag_labels)
    
    label_file_name = '/Users/sumeet_kumar/Library/CloudStorage/OneDrive-IndianSchoolofBusiness/Research_projects/Stance/Stance/stance_dataset/labeled_users/' + event_name  + '_labels_combined.txt' #If not, leave blank    
    
    event_data = DataTablesExtractor.get_event_data(all_events = [event_name], # 'open_us',
                                 output_path = results_path, #timeline
                                 data_folder = extracted_files_path ,
                                label_folder_path = '/Users/sumeet_kumar/Library/CloudStorage/OneDrive-IndianSchoolofBusiness/Research_projects/Stance/Stance/stance_dataset/labeled_users/',
                               label_file_extension =  '_labels_combined.txt' ,
                               use_labeled_users_only = False)    #




    # Set the parameters for the model
    print(seed_tag_labels, label_file_name, event_name )
    dataJson = event_data[event_name] # loadBiasWatchJson(folder_path, text_file_name, label_file_name )
    
    
    args = {}
    args['binarize'] = False
    args['retweet_count'] = 5000
    args['hashtag_count'] = 250#250
    args['inner_iteration_count']  = 5
    args['iteration_count'] = 3
    args['property_type'] = 2

    args['text_threshold'] = 0.85 # 0.65         
    args['property_confidence_threshold'] = 0.8
    args['confidence_threshold_for_mixing'] = 0.7
    mixing_percent = args['mixing_percent'] = 0.05

    args['seed_tag_labels'] = seed_tag_labels        
    args['name'], args['clf'], args['parameters'] =  ('svm12', Pipeline([('vect', CountVectorizer()),
                                         ('tfidf', TfidfTransformer()),
                                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                                                   alpha=1e-3, max_iter=15, random_state=42)),
                                        ])
                                    , {'vect__ngram_range': [(1, 1), (1, 2)], 
                                      'tfidf__use_idf': (True, False),
                                      'clf-svm__alpha': (1e-2, 1e-3),
                                     })

    args['experiment_type'] = event_name
    args['dataJson'] = dataJson        

    args['results_path'] = results_path
    
    
    print(dataJson)


    # Run the model that creates output files
#     predicted_user_stance, predicted_user_stance_conf = run_cotrain_experiments(args)


#     event_predicted_user_stance[event_name] = predicted_user_stance
    
#     event_predicted_user_stance_conf[event_name] = predicted_user_stance_conf    
    event_table_data[event_name] = event_data
    
    

{'defendthesecond': 1, '2ashallnotbeinfringed': 1, 'guncontrolnow': -1, 'gunreformnow': -1, 'marchforourlives': -1, '2adefenders': 1, '2ndamendment': 1}
error endtag count
error user count
error endtag count
error source_tweet_user count
error retweet_user count
error source_user count
error source_user count
error reply_user count
user_tweet_count:  55813
user_retweet_count:   4382
user_endtag_count:  52375
endtag_count:  17821
user_endtag_count examples:  ['user', '1001572752', '2496463412', '823245747818733568', '2819902607', '3214757596', '41628696', '3267456386', '887442796268978176', '959797135158935553', '939899077927620614', '827373737112641537', '112296882', '73408582', '29519474', '795099402926956544', '968250635299602432', '1385420359', '15410674', '17633850']
user_retweet_count exampless:  ['source_tweet_user', '60912576', '317385247', '604940737', '289342771', '14216661', '14435027', '3250141399', '874636442290577410', '14629315', '5988062', '202952121', '30726383', '13675

In [4]:
event_table_data.keys()

dict_keys(['Student_Marches', 'Santa_Fe_Shooting', 'Iran_Deal'])

In [31]:
df = pd.read_csv('./data/df_all_data.csv', dtype={'target_user': object, 'response_user': object})

users_of_interest = {}
for user in df['target_user']:
    users_of_interest[user] = 1

print(len(users_of_interest))    

for user in df['response_user']:
    users_of_interest[user] = 1
print(len(users_of_interest))        



1092
3347


In [34]:
# self.user_retweet_count = user_retweet_count
#         self.user_endtag_count = user_endtag_count


from os import path

# all_events = ['Santa_Fe_Shooting'] # , 'Student_Marches' , 'Iran_Deal'
# label_folder_path = '../stance_dataset/contentious_data/'
# data_folder = '../stance_dataset/contentious_data/'
# label_file_extension = '/endtags_weighted.csv'


def add_user_feature_count_to_dict(user_dict, user, feature, count):
        try:
            if user not in user_dict:
                user_dict[user] = {}

            user_feature_count =  user_dict[user]

            if feature not in user_feature_count:
                user_feature_count[feature] = int(count)
            else:
                user_feature_count[feature] += int(count)
        except:
            print('error', feature, count)

def add_feature_count_to_dict(feature_dict, feature, count):
    try:
        if feature not in feature_dict:
            feature_dict[feature] = int(count)
        else:
            feature_dict[feature] += int(count)

    except:
        print('error', feature, count)


    
def create_vector(user_property_count, tweet_user_index, N_TOP, binarize_count):
    user_hashtag_weighted_matrix = np.zeros((len(tweet_user_index), N_TOP))

    hashtags_users = {}
    hashtag_index = {}
    index_to_tag = {}
    for user, hashtags_count in user_property_count.items():
        for hashtag, count in hashtags_count.items():
            if hashtag not in hashtags_users:
                hashtags_users[hashtag] = []

            hashtags_users[hashtag].append(user)


    print('hashtags_users: ', len(hashtags_users))

    sorted_hashtags = sorted(hashtags_users.items(), key=lambda x: len(x[1]), reverse=True)

    for tag, users in sorted_hashtags[0:N_TOP]:
        for user in users:
            if user in tweet_user_index:
                user_index = tweet_user_index[user]
                if tag not in hashtag_index:
                    hashtag_index[tag] = len(hashtag_index)
                    index_to_tag[len(hashtag_index) - 1] = tag

                tag_index = hashtag_index[tag]

                if tag in user_property_count[user]:
                    user_hastag_time_used = user_property_count[user][tag]

                    if binarize_count:
                        user_hashtag_weighted_matrix[user_index, tag_index] = 1
                    else:
                        user_hashtag_weighted_matrix[user_index, tag_index] = user_hastag_time_used

#     if normalize_row:
#         normalized_user_hashtag_weighted_matrix = TextProcessingHelpers.row_normalize(user_hashtag_weighted_matrix)
#     else:
    normalized_user_hashtag_weighted_matrix = user_hashtag_weighted_matrix

    return normalized_user_hashtag_weighted_matrix, hashtag_index, index_to_tag


tweet_user_index = {}
tweet_user_index_to_id = {}

user_retweet_count_all= {}
user_endtag_count_all = {}
for event_name in ['Student_Marches', 'Santa_Fe_Shooting', 'Iran_Deal']: # ]:

    # user_tweet_count = event_table_data['Student_Marches']['Student_Marches'].user_tweet_count
    user_retweet_count = event_table_data[event_name][event_name].user_retweet_count
    user_endtag_count = event_table_data[event_name][event_name].user_endtag_count

    for user, count in user_endtag_count.items():
        if (user not in tweet_user_index and sum(count.values()) > 5) or (user not in tweet_user_index and user in users_of_interest):
            tweet_user_index_length = len(tweet_user_index)
            tweet_user_index[user] = tweet_user_index_length
            tweet_user_index_to_id[tweet_user_index_length] = user


    for user, count in user_retweet_count.items():
        if (user not in tweet_user_index and sum(count.values()) > 5) or (user not in tweet_user_index and user in users_of_interest):
            tweet_user_index_length = len(tweet_user_index)
            tweet_user_index[user] = tweet_user_index_length
            tweet_user_index_to_id[tweet_user_index_length] = user
            
    user_retweet_count_all = {**user_retweet_count_all, **user_retweet_count}
    user_endtag_count_all = {**user_endtag_count_all, **user_endtag_count}
            
            

print('Creating hashtag weight matrix')
user_hashtag_weighted_matrix, hashtag_index, index_to_tag = create_vector(user_endtag_count,
                                                                          tweet_user_index,
                                                                          N_TOP = 100,
                                                                          binarize_count = False)


user_retweet_weighted_matrix, retweet_index, index_to_retweet = create_vector(user_retweet_count,
                                                                          tweet_user_index,
                                                                          N_TOP = 100,
                                                                          binarize_count = False)



print(np.shape(user_hashtag_weighted_matrix))

print(np.shape(user_retweet_weighted_matrix))

# Creating hashtag weight matrix
# hashtags_users:  86653
# hashtags_users:  580180
# (70560, 100)
# (70560, 100)

Creating hashtag weight matrix
hashtags_users:  86653
hashtags_users:  580180
(71293, 100)
(71293, 100)


In [35]:

# sum(user_endtag_count['2496463412'].values())
# np.sum(user_hashtag_weighted_matrix, axis=1)
# np.sum(user_retweet_weighted_matrix, axis=1)
# print(np.shape(user_hashtag_weighted_matrix))

user_user_matrix = np.dot(user_hashtag_weighted_matrix, user_hashtag_weighted_matrix.T)

print(np.shape(user_user_matrix))

(71293, 71293)


In [36]:
user_user_matrix2 = np.dot(user_retweet_weighted_matrix, user_retweet_weighted_matrix.T)

print(np.shape(user_user_matrix2))

(71293, 71293)


In [37]:
user_user_matrix_joint = user_user_matrix + user_user_matrix2

print(np.shape(user_user_matrix_joint))

(71293, 71293)


In [49]:
np.save('./data/user_user_matrix_joint.npy', user_user_matrix_joint)

In [38]:
import pandas as pd
from stellargraph import StellarGraph
import numpy as np
import pandas as pd
import os

import stellargraph as sg
# from stellargraph.mapper import FullBat

k = 0
I, J = np.shape(user_user_matrix_joint)

sources = []
targets = []
weights = []
indicies = []

# square_node_data = pd.DataFrame(
#     {"x": [1, 2, 3, 4], "y": [-0.2, 0.3, 0.0, -0.5]}, index=["a", "b", "c", "d"]
# )
# square_node_data

user_features= {}
for i in range(0, I):
    for j in range(0, J):

        if i!= j and user_user_matrix[i, j] > 0:
            user1 = tweet_user_index_to_id[i]
            sources.append(user1)
            
#             if user1 not in user_features:
#                 user_features[user1] = np.zeros(10).tolist()
                
            user2 = tweet_user_index_to_id[j]
            targets.append(user2)
            
#             if user2 not in user_features:
#                 user_features[user2] = np.zeros(10).tolist()
            
            
            weights.append(user_user_matrix[i, j])

            k += 1
        
print('k: ', k, len(user_features))   

event_edges = pd.DataFrame(
    {"source": sources, "target": targets, "weight":weights }
)

G = StellarGraph(edges= event_edges)
print(G.info())

k:  1657469888 0
StellarGraph: Undirected multigraph
 Nodes: 54121, Edges: 1657469888

 Node types:
  default: [54121]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [1657469888]
        Weights: range=[1, 2.49968e+06], mean=40.1005, std=3026.22
        Features: none


In [39]:
from stellargraph.data import BiasedRandomWalk

rw = BiasedRandomWalk(G)

walks = rw.run(
    nodes=list(G.nodes()),  # root nodes
    length=10,  # maximum length of a random walk
    n=5,  # number of random walks per root node
    p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
    q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
)
print("Number of random walks: {}".format(len(walks)))




Number of random walks: 270605


In [40]:
from gensim.models import Word2Vec

str_walks = [[str(n) for n in walk] for walk in walks]
model = Word2Vec(str_walks, vector_size=128, window=5, min_count=0, sg=1, workers=4)
# The embedding vectors can be retrieved from model.wv using the node ID.




In [41]:

# Retrieve node embeddings and corresponding subjects
node_ids = model.wv.index_to_key  # list of node IDs
node_embeddings = (
    model.wv.vectors
)  # numpy.ndarray of size number of nodes times embeddings dimensionality


print(len(node_ids), type(node_embeddings))

54121 <class 'numpy.ndarray'>


In [42]:
model.wv.save_word2vec_format('./data/node_embeddings_128_v2.text', binary=False)

# from gensim.models import KeyedVectors

# save_word2vec_format()

# kv = KeyedVectors(128)

# kv.add(node_ids, node_embeddings)
# kv.save('./data/node_embeddings_128.kvmodel')
# # You could then later re-load these via:



In [24]:
# kv2 = KeyedVectors.load('../data/node_embeddings_128.kvmodel')

FileNotFoundError: [Errno 2] No such file or directory: '../data/node_embeddings_128.kvmodel'

In [43]:

model1 = Word2Vec(str_walks, vector_size=64, window=5, min_count=0, sg=1, workers=4)


In [44]:
model1.wv.save_word2vec_format('./data/node_embeddings_64_v2.text', binary=False)

In [45]:

model2 = Word2Vec(str_walks, vector_size=16, window=5, min_count=0, sg=1, workers=4)


In [46]:
model2.wv.save_word2vec_format('./data/node_embeddings_16_v2.text', binary=False)

In [47]:

model3 = Word2Vec(str_walks, vector_size=8, window=5, min_count=0, sg=1, workers=4)


In [48]:
model2.wv.save_word2vec_format('./data/node_embeddings_8_v2.text', binary=False)

In [17]:
len(node_ids)

53847

In [ ]:
np.shape(node_embeddings)

In [ ]:

node_targets = node_subjects.loc[[int(node_id) for node_id in node_ids]]

transform = TSNE  # PCA

trans = transform(n_components=2)
node_embeddings_2d = trans.fit_transform(node_embeddings)


# draw the embedding points, coloring them by the target label (paper subject)
alpha = 0.7
# label_map = {l: i for i, l in enumerate(np.unique(node_targets))}
# node_colours = [label_map[target] for target in node_targets]

plt.figure(figsize=(7, 7))
plt.axes().set(aspect="equal")
plt.scatter(
    node_embeddings_2d[:, 0],
    node_embeddings_2d[:, 1],
#     c=node_colours,
    cmap="jet",
    alpha=alpha,
)
plt.title("{} visualization of node embeddings".format(transform.__name__))
plt.show()